In [ ]:
! pip install colour
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import datetime
import sklearn
import os
from colour import Color
from matplotlib import dates as mpl_dates
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Bidirectional
from sklearn import metrics

color = sns.color_palette()
file_path = "/content/drive/MyDrive/"
def load_data(file_path, file_name):
  df = pd.read_csv(file_path+file_name, parse_dates=["date"])
  #df = df[df['DATA']> start]
  df.head()
  return df
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=3

test = 'ssr-chi2test'

def grangers_causality_matrix(data, variables, test = 'ssr_chi2test', verbose=False):

    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)

    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')

            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value

    dataset.columns = [var + '_x' for var in variables]

    dataset.index = [var + '_y' for var in variables]

    return dataset

def grangers_causality_vector(data, variables,target = 'close', test = 'ssr_chi2test', verbose=False):

    dataset = pd.DataFrame(columns=variables)
    datas = []
    for c in variables:
      test_result = grangercausalitytests(data[[target,c]].dropna(), maxlag=maxlag, verbose=False)
      p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
      if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
      min_p_value = np.min(p_values)
      dataset.loc[0,c] = min_p_value
    # for c in dataset.columns:
    #     for r in dataset.index:
    #         test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
    #         p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
    #         if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')

    #         min_p_value = np.min(p_values)
    #         dataset.loc[r,c] = min_p_value

    # dataset.columns = [var + '_x' for var in variables]

    # dataset.index = [var + '_y' for var in variables]

    return dataset
def calc_corr(df, thr,do_plot=1):
  corrmat = df.corr(method='pearson')
  # fig, ax = plt.subplots(figsize=(20, 20))
  
  red = Color("red")
  colors = list(red.range_to(Color("green"),len(df.columns)))
  colors = [color.rgb for color in colors]

  corrmat = corrmat.sort_values(by=['close'])
  # corrmat['close'].plot.barh(color=colors)
  #sns.heatmap(corrmat, vmax=1., square=False)
  corrmat = corrmat[(corrmat['close'] > thr)]
  selected_features = corrmat.index
  if do_plot:
    plt.grid(True, which='both', axis='x')
    plt.title("Cryptocurrency correlation map", fontsize=15)
    plt.savefig("correlation map")
    plt.show()
    plt.close()

  return list(selected_features)
def split_data(df, seq_size, selected_features, target):
    num_of_slices = int(len(df) / seq_size)
    
    X = list()
    Y = list()
    for i in range(0,len(df)-seq_size):
        sequence = df.iloc[i:i+seq_size - 1][selected_features].values
        # if len(X) == 0:
        #   X = sequence
        #   Y = df.iloc[i+seq_size][target]
        # else:
        X.append(sequence)
        Y.append(df.iloc[i+seq_size][target]) 
        # if len(X) == 0:
        #     X = sequence
        # else:
        #     X= 
        #     #rms_res = np.concatenate((rms_res, rms_window), axis=0)
        # trg.append(rms[i + window_size])
        #     #rul.append(int((1 - (((j * size)+i)/life))*10))
    return X, Y

def normalize(X, Y,normalise_type=0):
  X_n = [[[0 for k in range(len(X[0][0]))] for j in range(len(X[0]))] for i in range(len(X))]
  Y_n = [0 for k in range(len(Y))]
  inverse_transform = [0 for k in range(len(Y))]
  for i in range(0, len(X)): 
    for j in range(0, len(X[0])): 
      for k in range (0, len(X[0][0])): 
        # if 'GDELT' in selected_features[k]:
        #   X_n[i][j][k] = X[i][j][k]
        # else:
        if normalise_type == 0:
          if X[i][0][k] != 0:
            X_n[i][j][k] = (X[i][j][k] / X[i][0][k]) - 1 
        if normalise_type == 1:
          if X[i][j-1][k] !=0:
            X_n[i][j][k] = (X[i][j][k] / X[i][j-1][k]) - 1 

  return X_n

from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Bidirectional
from sklearn import metrics
def LSTM_net(X, Y, in_shape, ep=200, val=0.3, ver=2):
    model = Sequential()
    model.add(LSTM(10,return_sequences=True, input_shape=in_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(10,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(10,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(10,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Dense(1))
   
    

    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    opt = tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
              optimizer=opt,
              metrics=["mse"])
    
    #model.compile(optimizer='adam', loss='mse')
    history = model.fit(X,Y, epochs=ep, validation_split=val, verbose=ver,callbacks=[lr_schedule])
    
    # print(history.history["loss"])
    # plt.semilogx(history.history["lr"], history.history["loss"])
    # plt.axis([1e-2, 0, 0, 0.1])   
    # plt.show()
    # plt.close()
    ind = np.argmin(history.history["loss"])
    
    LR = history.history["lr"][ind]
    LR= 0.001
    # print('Learning_rate: ' , LR)
    model.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(learning_rate= LR))#tf.keras.optimizers.RMSprop())
    history = model.fit(X, Y, epochs=ep, validation_split=val, verbose=ver, shuffle=False)

    
    return model
    
import random
import numpy as np
from collections import deque

import torch
import torch.nn as nn
import torch.autograd as autograd
# df = load_data(file_path, "data_set.csv")
# df = load_data(file_path, "new_dataset.csv")
def risk_sensitive_func(x,a,mode,b=10):
  if mode == 'Pragmatism':
    y = 1-np.exp(-a*abs(x-1))
    return y
  if mode == 'Threshold':
    y = 1-np.exp(-a*abs(x-1))
    if y>0.5:
      k = 1
    else:
      k = 0
    return k
  if mode == 'RiskLevel':
    a1 = a
    a2 = b/a
    if x>=1:
      y = 1-np.exp(-a1*abs(x-1))
    if x<1:
      y = 1-np.exp(-a2*abs(x-1))
    return y
  if mode == 'Thresholdrisk':
    a1 = a
    a2 = b/a
    if x>=1:
      y = 1-np.exp(-a1*abs(x-1))
    if x<1:
      y = 1-np.exp(-a2*abs(x-1))
    if y>0.5:
      k = 1
    else:
      k = 0
    return k
def get_risk_sensitive_params(mode):
  if mode == 'Pragmatism':
    y = [[5],[10],[40],[80],[160]]
    return y
  if mode == 'Threshold':
    y = [[5],[10],[40],[80],[160]]
    return y
  if mode == 'RiskLevel':
    y = [[1,80],[10,80],[20,80],[40,80],[60,80],[80,80]]
    # b = [10,10,10,40,40,40]
    return y
  if mode == 'Thresholdrisk':
    y = [[1,80],[10,80],[20,80],[40,80],[60,80],[80,80]]
    # b = [10,10,10,40,40,40]
    return y

#@title Control Panel
dataset_type = "hourly" #@param ["daily", "hourly"]
use_loss_limit = False #@param {type:"boolean"}
profiting_interval = False #@param {type:"boolean"}
crypto_type = "eth" #@param ["btc", "ltc", "bnb", "eth"]
observe_delay = "_2" #@param ["", "_2", "_3", "_6","_12"] {type:"string"}

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Task1 

In [ ]:
import warnings
import re,os
import pandas as pd
import numpy as np
import keras

import datetime as dt
import dateutil.relativedelta as relativedelta
from datetime import datetime
import re
from scipy.signal import argrelextrema
warnings.filterwarnings('ignore')
# #@title Control Panel
# dataset_type = "daily" #@param ["daily", "hourly"]
# use_loss_limit = False #@param {type:"boolean"}
# profiting_interval = False #@param {type:"boolean"}
# crypto_type = "btc" #@param ["btc", "ltc", "bnb", "eth"]
# observe_delay = "_2" #@param ["", "_2", "_3", "_6","_12"] {type:"string"}
round_cnt = 0
do_the_new_idea = False
redu_prev_trades = False
redu_previous_preds = False
ignore_existense_trades = True
dont_sell_inside_candle = True
Transaction_cost_rate = 0.001
# dont_sell_inside_candle = False
# for crypto_type in ["btc"]: #["btc", "ltc", "eth"]:
for crypto_type in ["eth"]:
  for mode_dataset_strip in ['remove_first_month','use_all_from_begin']:
    for dataset_type in ['daily','hourly']:
    # for dataset_type in ['hourly']:
      
      if dataset_type == 'daily':
        list_options = ["", "_2", "_3"]
        list_options = [""]
      if dataset_type == 'hourly':
        list_options = ["","_3" ,"_6", "_12"]
        list_options = ["_3" ,"_6", "_12"]
        # list_options = [""]

      for observe_delay in list_options:
        dataset_name = 'Crypto_Datasets_shared/'+crypto_type+'_'+dataset_type+str(observe_delay)
          
        round_cnt+=1
        print('round ',round_cnt)
        print('mode_dataset_strip ' ,mode_dataset_strip)
        print('dataset_type ',dataset_type)
        print('observe_delay ',observe_delay)
        # if not profiting_interval:
        #   end_time = '2021-07-01 00:00:00'
        # if profiting_interval:
        #   end_time = '2020-12-31 00:00:00'
        file1 = '/content/drive/MyDrive/long_term_simulation_with_loss_bound_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv'
        file2 = '/content/drive/MyDrive/long_term_simulation_without_loss_bound_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv'
        do_prediction_in_this_loop = False
        do_trades_in_this_loop = False
        if os.path.exists(file1) and os.path.exists(file2) and not ignore_existense_trades:
          print('skipping as it has already runned completely')
          if redu_prev_trades:
              do_trades_in_this_loop = True
        else:
          do_trades_in_this_loop = True
          if not redu_previous_preds and  os.path.exists('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv'):
              print('resume incomplete process')
              do_prediction_in_this_loop = True
              df_predictions_hourly = pd.read_csv('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv')     
              
              if dataset_type == 'daily':
                  df_predictions_hourly['date']= pd.to_datetime(df_predictions_hourly['date'],format='%Y-%m-%d', errors='coerce').dt.date
              else:
                  df_predictions_hourly['date']= pd.to_datetime(df_predictions_hourly['date'],format='%Y-%m-%d %H:%M:%S', errors='coerce')
              last_test_date = df_predictions_hourly['date'][len(df_predictions_hourly)-1] - pd.DateOffset(days=round(1))
              end_train_time = str(last_test_date)
              print('end_train_time = ',str(last_test_date))
              ender_D  = df_predictions_hourly['date'][len(df_predictions_hourly)-1]  + pd.DateOffset(days=round(30))
              ender_time = str(ender_D)
              print('ender_time = ',str(ender_D))
              list_time_stomps_lstm = df_predictions_hourly['date'].tolist()
              list_time_stomps_lstm_attention = []
              list_perdictions_lstm = list(df_predictions_hourly['Estimated_change_ratio_LSTM'])
              list_perdictions_lstm_attention = []
              
          else:
              do_prediction_in_this_loop = True
              end_train_time = '2018-06-30 00:00:00'
              ender_time = '2018-08-01 00:00:00'
              df_predictions_hourly = pd.DataFrame()
              list_time_stomps_lstm = []
              list_time_stomps_lstm_attention = []
              list_perdictions_lstm = []
              list_perdictions_lstm_attention = []
        
        
          
          if do_prediction_in_this_loop:
            print('starting the predictions')
            dd = datetime.now()
            created_date = str(dd.year)+'-'+str(dd.month)+'-'+str(dd.day)+' -H'+str(dd.hour)
            df = load_data(file_path, dataset_name+".csv")
            # rd = relativedelta.relativedelta(day = 6)
            start_time = str(df.loc[0,'date'])
    
            # start_time = '2017-01-01 00:00:00'
            '''
            if os.path.exists('/content/drive/MyDrive/inference_hourly.csv'):
              df_predictions_hourly = pd.read_csv('/content/drive/MyDrive/inference_hourly.csv')   
              start_time = df_predictions_hourly.loc[len(df_predictions_hourly)-1,'date']
            '''
            end_time = str(df.loc[len(df)-1,'date'])
            # end_time = '2021-08-08 00:00:00'
            text = start_time
            if dataset_type == 'daily':
              match = re.search(r'\d{4}-\d{2}-\d{2}', text)
            else:
              match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', text)
            if dataset_type == 'daily':
              start_date = datetime.strptime(match.group(), '%Y-%m-%d').date()
            else:
              start_date = datetime.strptime(match.group(), '%Y-%m-%d %H:%M:%S')
    
            text = end_train_time
            if dataset_type == 'daily':
              match = re.search(r'\d{4}-\d{2}-\d{2}', text)
            else:
              match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', text)
            if dataset_type == 'daily':
              end_train_date = datetime.strptime(match.group(), '%Y-%m-%d').date()
            else:
              end_train_date = datetime.strptime(match.group(), '%Y-%m-%d %H:%M:%S')
    
            text = end_time
            if dataset_type == 'daily':
              match = re.search(r'\d{4}-\d{2}-\d{2}', text)
            else:
              match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', text)
            # =========== if match is not None:
            if dataset_type == 'daily':
              end_date = datetime.strptime(match.group(), '%Y-%m-%d').date()
            else:
              end_date = datetime.strptime(match.group(), '%Y-%m-%d %H:%M:%S')
            # =========== if match is not None:
            # start_date2 = datetime.strptime(match.group(), '%Y-%m-%d').date()
            text = ender_time
            if dataset_type == 'daily':
              match = re.search(r'\d{4}-\d{2}-\d{2}', text)
            else:
              match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', text)
            # =========== if match is not None:
            if dataset_type == 'daily':
              ender_date = datetime.strptime(match.group(), '%Y-%m-%d').date()
            else:
              ender_date = datetime.strptime(match.group(), '%Y-%m-%d %H:%M:%S')
            # start_date2 =ender_date-pd.DateOffset(days=round(562))
            if mode_dataset_strip == 'remove_first_month':
              start_date2 =ender_date-pd.DateOffset(days=round(375))
            else:
              start_date2 = start_date
            # start_date2 =ender_date-pd.DateOffset(days=round(210))
            # tempdate = start_date2 
            
            cnt = 0
            
            break_next_time = 0
            num_days = 360
            while True:
              
              if ender_date > end_date:
                break
              #print(list_time_stomps_lstm)
              #print(list_time_stomps_lstm_attention)
              #print(list_perdictions_lstm)
              #print(list_perdictions_lstm_attention)
              cnt+=1
              
    
              do_mapping = False
              from datetime import datetime
              import re
              from scipy.signal import argrelextrema
              
              if dataset_type == 'daily':
                match = re.search(r'\d{4}-\d{2}-\d{2}', str(ender_date))
              else:
                match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(ender_date))
              # if match is not None:
              # end_date = datetime.strptime(match.group(), "%Y-%m-%d")
              # start_date.month = 11
              df = load_data(file_path, dataset_name+".csv")
              df = df.drop(columns=['symbol'])
              # print(df['date'])
              # self.df["date"] = pd.to_datetime(self.df["date"]).dt.date
              if dataset_type == 'daily':
                df['date']= pd.to_datetime(df['date'],format='%Y-%m-%d', errors='coerce').dt.date
              else:
                df['date']= pd.to_datetime(df['date'],format='%Y-%m-%d %H:%M:%S', errors='coerce')
              df['close2'] = df['close']
              data_bitcoin2015 = df.loc[(start_date2<=df['date'])&(df['date']<=ender_date)]
              data_bitcoin2015 = data_bitcoin2015.reset_index(drop=True)
              data_bitcoin2015 = data_bitcoin2015.drop(columns = ['Unnamed: 0'])
              data_bitcoin2015_normalized = data_bitcoin2015.copy()
              for col in data_bitcoin2015_normalized.columns:
                data_bitcoin2015_normalized[col].fillna(method='ffill', inplace=True)
              num_rows = len(data_bitcoin2015_normalized)
              columns = list(data_bitcoin2015_normalized.columns)
              for c in columns:
                if 'GDELT' in c and not 'Tone' in c:
                  data_bitcoin2015_normalized[c] = pd.to_numeric(data_bitcoin2015_normalized[c], downcast="float")
                  for ic in range(len(data_bitcoin2015_normalized)-1):
                    data_bitcoin2015_normalized.loc[ic+1,c] = data_bitcoin2015_normalized.loc[ic+1,c]*0.1+0.9*data_bitcoin2015_normalized.loc[ic,c]
              df = data_bitcoin2015_normalized.copy()
    
    
    
              # order: How many points on each side to use for the comparison to consider
              ilocs_min = argrelextrema(data_bitcoin2015.close.values, np.less_equal, order=15)[0]
              ilocs_max = argrelextrema(data_bitcoin2015.close.values, np.greater_equal, order=15)[0]
    
              # df.Close.plot(figsize=(20,8), alpha=.3)
              # # filter prices that are peaks and plot them differently to be visable on the plot
              # data_bitcoin2015.iloc[ilocs_max].Close.plot(style='.', lw=10, color='red', marker="v");
              # data_bitcoin2015.iloc[ilocs_min].Close.plot(style='.', lw=10, color='green', marker="^");
              if do_mapping:
                num_cols = len(columns)
                for col in range(2,num_cols,1):
                  for row in range(1,num_rows,1):
                    if columns[col]=='close':
                      if data_bitcoin2015.loc[row-1, columns[col]] == 0:
                        data_bitcoin2015_normalized.loc[row, columns[col]] = 1#data_bitcoin2015_normalized.loc[row-1, columns[col]]  
                      else:
                        if data_bitcoin2015.loc[row, columns[col]]/data_bitcoin2015.loc[row-1, columns[col]]>1:
                          data_bitcoin2015_normalized.loc[row, columns[col]] = 1.5
                        else:
                          data_bitcoin2015_normalized.loc[row, columns[col]] = 0.5
              else:
                num_cols = len(columns)
                for col in range(2,num_cols,1):
                  for row in range(1,num_rows,1):
                    if columns[col]=='close':
                      if data_bitcoin2015.loc[row-1, columns[col]] == 0:
                        data_bitcoin2015_normalized.loc[row, columns[col]] = data_bitcoin2015_normalized.loc[row-1, columns[col]]  
                      else:
                        data_bitcoin2015_normalized.loc[row, columns[col]] = 1+5*(data_bitcoin2015.loc[row, columns[col]]/data_bitcoin2015.loc[row-1, columns[col]]-1)
              # data_bitcoin2015_normalized.remove('Unnamed: 0')
              # data_bitcoin2015_normalized = data_bitcoin2015_normalized.drop(columns = ['Unnamed: 0'])
    
              data_bitcoin2015_normalized_rl = data_bitcoin2015.copy()
              columns = list(data_bitcoin2015_normalized_rl.columns)
              num_rows = len(data_bitcoin2015_normalized_rl)
              num_cols = len(columns)
              # for col in range(2,num_cols,1):
              #   for row in range(2,num_rows,1):
              #     if data_bitcoin2015.loc[row-1, columns[col]] != 0:
              #       data_bitcoin2015_normalized_rl.loc[row, columns[col]] = data_bitcoin2015.loc[row, columns[col]]/data_bitcoin2015.loc[row-1, columns[col]]
              #     else: 
              #       data_bitcoin2015_normalized_rl.loc[row, columns[col]] = 1
              # data_bitcoin2015_normalized_rl = data_bitcoin2015_normalized_rl.fillna(method='ffill')
    
              import pickle
              MAX_EPISODES = 3
              import os
              BATCH_SIZE = 32
              num_runs = 10
              Risk_modes = ['Thresholdrisk','Pragmatism','Threshold','RiskLevel']
              method_list = ['LSTM'] #['LSTM_self_Attention','LSTM']
              feature_engineering_method_list = ['Corr']#,'Granger']
              risk_sensitive_params = [1,2,5,10,40]
              corr_threshold_list = [0.7,0.8,0.85] #[0,0.2,0.4,0.6,0.7,0.75,0.8,0.85,9]
              split_percentage_list = [0.84] # [0.75,0.8,0.85,0.9]
              seq_size_list = [10]
              normalise_list = [1]
              days_after_to_predict_list = [1]
              #,3,5,7]
              max_mse = [200000,200000,200000,200000]
              min_class_acc = [0,0,0,0]
              has_runned_percentage_method = [[False,False,False],[False,False,False],[False,False,False],[False,False,False]]
              DDQN_runned = [[False,False,False],[False,False,False],[False,False,False],[False,False,False]]
              DDQN_LSTM_runned = [[False,False,False],[False,False,False],[False,False,False],[False,False,False]]
              runned_before = 0
              #  np.zeros((len(days_after_to_predict_list),len(split_percentage_list),len(method_list)))
              df_results = pd.DataFrame()
              # dataset_name = 'Ethereum dataset'
              # df_results  = pd.read_pickle('/content/drive/MyDrive/Mofid_Ether_infer/result_'+dataset_name+'.pkl')
              # df_results.index= [i for i in range(len(df_results))]
              # pd.set_option('display.max_rows', len(df_results))
              num_r = 0 
    
              for days_after_to_predict in days_after_to_predict_list:
                for split_percentage in split_percentage_list:
                  for method in method_list:   
                    for feature_engineering_method in feature_engineering_method_list:
                      # for corr_threshold in corr_threshold_list:
                      for normalise_t in normalise_list:
                        for seq_size in seq_size_list:
                          num_r+=1
                          if num_r>runned_before:
                            
                            #print('method = ',method)
                            #print('days_after_to_predict = ',days_after_to_predict)
                            #print('feature engineering method = ',feature_engineering_method)
                            # print('corr threshold = ',corr_threshold)
                            #print('split_percentage = ',split_percentage)
                            #print('normalise_t = ',normalise_t)
                            #print('seq_size = ',seq_size)
                            # if not os.path.exists('/content/drive/MyDrive/Mofid_infer/'+end_time+'RS_'+risk_mode+'_'+str(risk_sensitive_param[0])+'_one_run'+str(split_percentage)+'_'+method+'_acc.pkl'):
                            corr_threshold = 0.8
                            if feature_engineering_method == 'Corr':
                              cols = df.columns
                              selected_features = []
                              #selected_features.extend(df.columns[2:])
                              selected_features.extend(calc_corr(df, corr_threshold,0))
                              selected_features.remove('close')
                              #print(selected_features)
                              
                              #display(df, selected_features)
                              num_of_features = len(selected_features)
                            if feature_engineering_method == 'Granger':
                              gc_df = grangers_causality_vector(df.iloc[:,2:], variables = df.iloc[:,2:].columns)  
                              selected_features = [c for c in gc_df.columns if 1>gc_df.loc[0,c]>corr_threshold]
                              num_of_features = len(selected_features)
                              #print(selected_features)
    
                            for col in data_bitcoin2015_normalized.columns:
                              data_bitcoin2015_normalized[col].fillna(method='ffill', inplace=True)
                            
                            df_use = data_bitcoin2015.loc[[i for j, i in enumerate(data_bitcoin2015.index) if j % days_after_to_predict == 0]]
                            df_use_norm = data_bitcoin2015_normalized.loc[[i for j, i in enumerate(data_bitcoin2015_normalized.index) if j % days_after_to_predict == 0]]
                            
                            #seq_size = input('Enter Sequence Size: ')
                            # df.loc[(start_date2<=df['date'])&(df['date']<=ender_date)]
                            time_to_delta = pd.DateOffset(days=round(0))
                            if dataset_type == 'daily':                          
                              if observe_delay == '':
                                time_to_delta = pd.DateOffset(days=round(seq_size))
                              if observe_delay == '_2':
                                time_to_delta = pd.DateOffset(days=round(seq_size*2))
                              if observe_delay == '_3':
                                time_to_delta = pd.DateOffset(days=round(seq_size*3))
                            if dataset_type == 'hourly':                          
                              if observe_delay == '':
                                time_to_delta = pd.DateOffset(hours=round(seq_size))
                              if observe_delay == '_6':
                                time_to_delta = pd.DateOffset(hours=round(seq_size*6))
                              if observe_delay == '_3':
                                time_to_delta = pd.DateOffset(hours=round(seq_size*3))
                              if observe_delay == '_12':
                                time_to_delta = pd.DateOffset(hours=round(seq_size*12))
                            df_train = df_use_norm.loc[(start_date2<=df_use_norm['date'])&(df_use_norm['date']<=end_train_date)]
                            df_test = df_use_norm.loc[(end_train_date-time_to_delta<=df_use_norm['date'])&(df_use_norm['date']<=ender_date)]
                            df_test_real_val = df_use.loc[(end_train_date-time_to_delta<=df_use['date'])&(df_use['date']<=ender_date)]
                            X_train, Y_train = split_data(df_train, seq_size ,1, 'close')
                            X_train = normalize(X_train, Y_train)
                            X_test, Y_test = split_data(df_test, seq_size ,selected_features, 'close')
                            X_test =  normalize(X_test, Y_test)
                            X_use ,_= split_data(df_use_norm, seq_size ,selected_features, 'close')
                            #print(len(X_use))
                            #print(len(df_use_norm))
                            X_train = np.array(X_train).reshape(len(X_train), seq_size-1, num_of_features)
                            Y_train = np.array(Y_train).reshape(len(Y_train), 1)
                            in_shape = (seq_size-1, num_of_features)
                            buy_sell_vec = []
                            buy_sell_vec_intensity = []
                            pred_use = [1 for i in range(seq_size-1)]
                            if method == 'buy_at_first':
                              buy_sell_vec = [1 for i in range(len(df_use))]
                              buy_sell_vec[0] = 1
                              buy_sell_vec_intensity = [1 for i in range(len(df_use))]
                            if method == 'LSTM':
                              best_acc = 0
                              n_retry = 0
                              best_out_test = []
                              while n_retry <3 and (n_retry >1 or best_acc <0.55):
                                for corr_threshold in corr_threshold_list:
                                  if feature_engineering_method == 'Corr':
                                    cols = df.columns
                                    selected_features = []
                                    #selected_features.extend(df.columns[2:])
                                    selected_features.extend(calc_corr(df, corr_threshold,0))
                                    selected_features.remove('close')
                                    #print(selected_features)
                                    
                                    #display(df, selected_features)
                                    num_of_features = len(selected_features)
                                  if feature_engineering_method == 'Granger':
                                    gc_df = grangers_causality_vector(df.iloc[:,2:], variables = df.iloc[:,2:].columns)  
                                    selected_features = [c for c in gc_df.columns if 1>gc_df.loc[0,c]>corr_threshold]
                                    num_of_features = len(selected_features)
                                    #print(selected_features)
    
                                  for col in data_bitcoin2015_normalized.columns:
                                    data_bitcoin2015_normalized[col].fillna(method='ffill', inplace=True)
                                  
                                  df_use = data_bitcoin2015.loc[[i for j, i in enumerate(data_bitcoin2015.index) if j % days_after_to_predict == 0]]
                                  df_use_norm = data_bitcoin2015_normalized.loc[[i for j, i in enumerate(data_bitcoin2015_normalized.index) if j % days_after_to_predict == 0]]
                                  
                                  #seq_size = input('Enter Sequence Size: ')
                                  # df.loc[(start_date2<=df['date'])&(df['date']<=ender_date)]
                                  df_train = df_use_norm.loc[(start_date2<=df_use_norm['date'])&(df_use_norm['date']<=end_train_date)]
                                  df_test = df_use_norm.loc[(end_train_date-time_to_delta<=df_use_norm['date'])&(df_use_norm['date']<=ender_date)]
                                  df_test_real_val = df_use.loc[(end_train_date-time_to_delta<=df_use['date'])&(df_use['date']<=ender_date)]
                                  X_train, Y_train = split_data(df_train, seq_size ,selected_features, 'close')
                                  X_train = normalize(X_train, Y_train)
                                  X_test, Y_test = split_data(df_test, seq_size ,selected_features, 'close')
                                  X_test =  normalize(X_test, Y_test)
                                  X_use ,_= split_data(df_use_norm, seq_size ,selected_features, 'close')
                                  #print(len(X_use))
                                  #print(len(df_use_norm))
                                  #print(len(X_train))
                                  #print(len(X_test))
                                  X_train = np.array(X_train).reshape(len(X_train), seq_size-1, num_of_features)
                                  Y_train = np.array(Y_train).reshape(len(Y_train), 1)
                                  in_shape = (seq_size-1, num_of_features)
                                  model = LSTM_net(X_train, Y_train, in_shape, ep =50, ver=0)
                                  Y_pred_rel = list()
                                  Y_real_rel = list()
                                  for i in range(len(X_test)):
                                    X_temp = np.array(X_test[i]).reshape(1, seq_size-1, num_of_features)
                                    Y_temp = model.predict(X_temp)
                                    Y_real_rel.append(Y_test[i])
                                    Y_pred_rel.append(Y_temp[0][-1])
                                  if np.any(np.isnan(Y_pred_rel)): 
                                    #print('y pred has nan')
                                    Y_pred_rel = [np.nan_to_num(citem) for citem in Y_pred_rel]  
                                  if np.any(np.isnan(Y_real_rel)):  
                                    #print('y real has nan')
                                    Y_real_rel = [np.nan_to_num(citem) for citem in Y_real_rel]
                                  if True: 
                                    mserror_rel = sklearn.metrics.mean_squared_error(Y_real_rel, Y_pred_rel)
                                    corrrr_rel = np.corrcoef(np.array(Y_real_rel).reshape((len(Y_real_rel))),np.array(Y_pred_rel).reshape((len(Y_real_rel))))[1,0]
                                    class_acc = np.mean(((np.array(Y_real_rel)>1)&(np.array(Y_pred_rel)>1))|((np.array(Y_real_rel)<1)&(np.array(Y_pred_rel)<1)))
                                    #print('classification_accuracy', class_acc)
                                    ts_pred = list()
                                    Y_pred = list()
                                    Y_real = list()
                                    for i in range(len(X_test)):
                                      X_temp = np.array(X_test[i]).reshape(1, seq_size-1, num_of_features)
                                      Y_temp = model.predict(X_temp)
                                      if i-1+len(df_train) in range(len(data_bitcoin2015)):
                                        Y_real.append(Y_test[i]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                        Y_pred.append(Y_temp[0][-1]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                        ts_pred.append(data_bitcoin2015.loc[i-1+len(df_train), 'date'])
                                      # Y_real.append(Y_test[i]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                      # Y_pred.append(Y_temp[0][-1]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                      # ts_pred.append(data_bitcoin2015.loc[i-1+len(df_train), 'date'])
                                    if np.any(np.isnan(Y_pred)): 
                                      #print('y pred has nan')
                                      Y_pred = [np.nan_to_num(citem) for citem in Y_pred]  
                                    if np.any(np.isnan(Y_real)):  
                                      #print('y real has nan')
                                      Y_real = [np.nan_to_num(citem) for citem in Y_real]
                                    mserror = sklearn.metrics.mean_squared_error(Y_real, Y_pred)
                                    # best_out_test = Y_pred_rel
                                    corrrr = np.corrcoef(np.array(Y_real).reshape((len(Y_real))),np.array(Y_pred).reshape((len(Y_real))))[1,0]
                                    #print(mserror)
                                    if class_acc>best_acc:
                                      temp_model= keras.models.clone_model(model)
                                      # model_copy.build((None, 10)) # replace 10 with number of variables in input layer
                                      # model_copy.compile(optimizer='rmsprop', loss='categorical_crossentropy')
                                      temp_model.set_weights(model.get_weights())
                                      best_acc = class_acc
                                      best_ts_pred = ts_pred
                                      best_out_test = Y_pred_rel 
                                  else:
                                    print('has nan') 
                                n_retry+=1
                                
                              cnt_ts = 0
                              for ts_item in best_ts_pred:
                                if not ts_item in list_time_stomps_lstm:
                                  list_time_stomps_lstm.append(ts_item)
                                  list_perdictions_lstm.append(best_out_test[cnt_ts][0])
                                cnt_ts+=1
                              
                              model= keras.models.clone_model(temp_model)
                              # model_copy.build((None, 10)) # replace 10 with number of variables in input layer
                              # model_copy.compile(optimizer='rmsprop', loss='categorical_crossentropy')
                              model.set_weights(temp_model.get_weights())
                              #print('classification_accuracy', best_acc)
                          #   #print(len(df_use_norm))
                          #   #print(len(pred_use))
    
                            if method == 'LSTM_self_Attention':
                              best_acc = 0
                              n_retry = 0
                              
                              while n_retry <4 and (n_retry >1 or best_acc <0.55):
                                n_retry+=1
                                model = LSTM_self_attention_net(X_train, Y_train, in_shape, ep =50, ver=0)
    
                                # Y_pred_rel = list()
                                # Y_real_rel = list()
                                # for i in range(len(X_use)):
                                #   X_temp = np.array(X_use[i]).reshape(1, seq_size-1, num_of_features)
                                #   Y_temp = model.predict(X_temp)
                                #   Y_real_rel.append(Y_test[i])
                                #   Y_pred_rel.append(Y_temp[0][-1])
                                #   pred_use.append(Y_temp[0][-1])
                                  
                                  
    
                                Y_pred_rel = list()
                                Y_real_rel = list()
                                for i in range(len(X_test)):
                                  X_temp = np.array(X_test[i]).reshape(1, seq_size-1, num_of_features)
                                  Y_temp = model.predict(X_temp)
                                  Y_real_rel.append(Y_test[i])
                                  Y_pred_rel.append(Y_temp[0][-1])
                              #     if Y_temp[0][-1]>1:
                              #       buy_sell_vec.append(1)
                              #     elif  Y_temp[0][-1]<1:
                              #       buy_sell_vec.append(2)
                              #     else:
                              #       buy_sell_vec.append(0)
                              #     if len(risk_sensitive_param)==1:
                              #       buy_sell_vec_intensity.append(risk_sensitive_func(Y_temp[0][-1],risk_sensitive_param[0],risk_mode))
                              #     if len(risk_sensitive_param)>1:
                              #       buy_sell_vec_intensity.append(risk_sensitive_func(Y_temp[0][-1],risk_sensitive_param[0],risk_mode,risk_sensitive_param[1]))
                                mserror_rel = sklearn.metrics.mean_squared_error(Y_real_rel, Y_pred_rel)
                                corrrr_rel = np.corrcoef(np.array(Y_real_rel).reshape((len(Y_real_rel))),np.array(Y_pred_rel).reshape((len(Y_real_rel))))[1,0]
                                class_acc = np.mean(((np.array(Y_real_rel)>1)&(np.array(Y_pred_rel)>1))|((np.array(Y_real_rel)<1)&(np.array(Y_pred_rel)<1)))
                                #print('classification_accuracy', class_acc)
                                ts_pred = list()
                                Y_pred = list()
                                Y_real = list()
                                for i in range(len(X_test)):
                                  X_temp = np.array(X_test[i]).reshape(1, seq_size-1, num_of_features)
                                  Y_temp = model.predict(X_temp)
                                  Y_real.append(Y_test[i]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                  Y_pred.append(Y_temp[0][-1]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                  ts_pred.append(data_bitcoin2015.loc[i-1+len(df_train), 'date'])
                                mserror = sklearn.metrics.mean_squared_error(Y_real, Y_pred)
                                corrrr = np.corrcoef(np.array(Y_real).reshape((len(Y_real))),np.array(Y_pred).reshape((len(Y_real))))[1,0]
                                #print(mserror)
                                if class_acc>best_acc:
                                  best_ts_pred = ts_pred
                                  best_out_test = Y_pred_rel
                                  temp_model= keras.models.clone_model(model)
                                  # model_copy.build((None, 10)) # replace 10 with number of variables in input layer
                                  # model_copy.compile(optimizer='rmsprop', loss='categorical_crossentropy')
                                  temp_model.set_weights(model.get_weights())
                                  best_acc = class_acc
                              cnt_ts = 0
                              for ts_item in best_ts_pred:
                                if not ts_item in list_time_stomps_lstm_attention:
                                  list_time_stomps_lstm_attention.append(ts_item)
                                  list_perdictions_lstm_attention.append(best_out_test[cnt_ts][0])
                                cnt_ts+=1
                              model= keras.models.clone_model(temp_model)
                              # model_copy.build((None, 10)) # replace 10 with number of variables in input layer
                              # model_copy.compile(optimizer='rmsprop', loss='categorical_crossentropy')
                              model.set_weights(temp_model.get_weights())
                              #print('classification_accuracy', best_acc)
                            #print(len(df_use_norm))
                            #print(len(pred_use))
                            if method == 'prev_day':
                              pred_use.append(1)
                              class_acc = 0
                              Y_pred_rel = list()
                              Y_real_rel = list()
                              for i in range(len(X_test)):
                                buy_sell_vec.append(random.randint(0,2))
                                # buy_sell_vec_intensity.append(risk_sensitive_func(random.random()*10-4,risk_sensitive_param[0],risk_mode))
                                if len(risk_sensitive_param)==1:
                                  buy_sell_vec_intensity.append(risk_sensitive_func(random.random()*10,risk_sensitive_param[0],risk_mode))
                                if len(risk_sensitive_param)>1:
                                  buy_sell_vec_intensity.append(risk_sensitive_func(random.random()*10,risk_sensitive_param[0],risk_mode,risk_sensitive_param[1]))
                                Y_pred_rel.append(1)
                                
                                Y_real_rel.append(Y_test[i])
                              Y_pred = list()
                              Y_real = list()
                              for i in range(len(X_test)):
                                if len(Y_real) >0:
                                  Y_pred.append(Y_real[-1])
                                else:
                                  Y_pred.append(1*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                                Y_real.append(Y_test[i]*data_bitcoin2015.loc[i-1+len(df_train), 'close'])
                              mserror = sklearn.metrics.mean_squared_error(Y_real, Y_pred)
                              corrrr = np.corrcoef(np.array(Y_real).reshape((len(Y_real))),np.array(Y_pred).reshape((len(Y_real))))[1,0]
                          df_predictions_hourly = pd.DataFrame()
                          df_predictions_hourly['date'] = list_time_stomps_lstm
                          df_predictions_hourly['Estimated_change_ratio_LSTM'] = list(np.array(list_perdictions_lstm))
                          df_predictions_hourly.loc[:,'Estimated_change_ratio_LSTM_Attention'] = pd.Series(list(np.array(list_perdictions_lstm)))
                          # df_predictions_hourly.to_csv('/content/drive/MyDrive/inference_daily_'+created_date+'.csv')                  
                          # if profiting_interval:
                          df_predictions_hourly.to_csv('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv')                
                        
              if mode_dataset_strip == 'remove_first_month':
                start_date2 =start_date2+pd.DateOffset(days=round(30))
              ender_date =ender_date+pd.DateOffset(days=round(30))
              end_train_date =end_train_date+pd.DateOffset(days=round(30))
            
            
            
            
            df_predictions_hourly = pd.DataFrame()
            df_predictions_hourly['date'] = list_time_stomps_lstm
            df_predictions_hourly['Estimated_change_ratio_LSTM'] = list(np.array(list_perdictions_lstm))
            df_predictions_hourly.loc[:,'Estimated_change_ratio_LSTM_Attention'] = pd.Series(list(np.array(list_perdictions_lstm)))
            # df_predictions_hourly.to_csv('/content/drive/MyDrive/inference_daily_'+created_date+'.csv')                  
            # if profiting_interval:
            df_predictions_hourly.to_csv('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv')                
            # if not profiting_interval:
            #   df_predictions_hourly.to_csv('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_non_profiting'+'.csv')   
        
        
        if do_trades_in_this_loop:
          print('starting the trading')
          if os.path.exists('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv'):
            df_predictions_hourly = pd.read_csv('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv')                
          else:
            print('doesnt exisst')
          # print(df_predictions_hourly.date)
  
          df = load_data(file_path, dataset_name+".csv")
          
          if dataset_type == 'daily':
            df['date']= pd.to_datetime(df['date'],format='%Y-%m-%d', errors='coerce')
          else:
            df['date']= pd.to_datetime(df['date'],format='%Y-%m-%d %H:%M:%S', errors='coerce')
          # print(df.date)
          df = df.loc[df['date'].isin(list(df_predictions_hourly['date']))]
          # df['date'] = df.loc[df['date'].isin(list(df_predictions_hourly['date']))]
          df.reset_index()
          df['close2'] = df['close']
          data_bitcoin2015 = df.copy()
          # data_bitcoin2015.remove('Unnamed: 0')
          data_bitcoin2015 = data_bitcoin2015.reset_index(drop=True)
          # data_bitcoin2015.remove('Unnamed: 0')
          # print(data_bitcoin2015.date)
          data_bitcoin2015_normalized = data_bitcoin2015.copy()
          for col in data_bitcoin2015_normalized.columns:
            data_bitcoin2015_normalized[col].fillna(method='ffill', inplace=True)
          
          num_rows = len(data_bitcoin2015_normalized)
          columns = list(data_bitcoin2015_normalized.columns)
          
          dd = datetime.now()
          created_date = str(dd.year)+'-'+str(dd.month)+'-'+str(dd.day)+' -H'+str(dd.hour)
          
          # if profiting_interval:
          
          # if not profiting_interval:
          #   if os.path.exists('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_profiting'+'.csv'):
          #     df_predictions_hourly = pd.read_csv('/content/drive/MyDrive/inference_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_non_profiting'+'.csv')                
          for use_loss_limit in [False, True]:  
            def sell_with_loss_limit(ld_oc,btc_price_open,btc_price_close,loss_limit,profit_limit,use_loss_limit=use_loss_limit):
              d_oc = btc_price_close-btc_price_open
              if d_oc>0:
                sell_price = btc_price_close
              if d_oc<=0:
                sell_price = max(btc_price_open-abs(d_oc)*loss_limit,btc_price_close)
              # return min(btc_price_open,btc_price_close)
              return btc_price_open
              if not use_loss_limit:
                return btc_price_close
              else:
                return sell_price
  
            def buy_with_loss_limit(ld_oc,btc_price_open,btc_price_close,loss_limit,profit_limit,use_loss_limit=use_loss_limit):
              d_oc = btc_price_close-btc_price_open
              if d_oc>0:
                buy_price = min(btc_price_open+abs(d_oc)*loss_limit,btc_price_close)
              if d_oc<=0:
                buy_price = btc_price_close
              return btc_price_open
              if not use_loss_limit:
                return btc_price_close
              else:
                return buy_price
              # return max(btc_price_open,btc_price_close)
              # return btc_price_close
              # return buy_price
            loss_bound = True
            loss_bound = False
            loss_limit = 0.01
            import pickle
            MAX_EPISODES = 3
            import os
            BATCH_SIZE = 32
            num_runs = 10
            Risk_modes = ['Thresholdrisk','Pragmatism','Threshold','RiskLevel']
            method_list = ['LSTM','buy_at_first','random']
            feature_engineering_method_list = ['Corr']#,'Granger']
            risk_sensitive_params = [1,2,5,10,40]
            corr_threshold_list = [0.8] #[0,0.2,0.4,0.6,0.7,0.75,0.8,0.85,9]
            split_percentage_list = [0.9] # [0.75,0.8,0.85,0.9]
            seq_size_list = [10]
            normalise_list = [1]
            days_after_to_predict_list = [1]
            #,3,5,7]
            max_mse = [200000,200000,200000,200000]
            min_class_acc = [0,0,0,0]
            has_runned_percentage_method = [[False,False,False],[False,False,False],[False,False,False],[False,False,False]]
            DDQN_runned = [[False,False,False],[False,False,False],[False,False,False],[False,False,False]]
            DDQN_LSTM_runned = [[False,False,False],[False,False,False],[False,False,False],[False,False,False]]
            runned_before = 0
            #  np.zeros((len(days_after_to_predict_list),len(split_percentage_list),len(method_list)))
            df_results = pd.DataFrame()
            # dataset_name = 'Crypto_Datasets/'+cbtc_daily'
            # df_results  = pd.read_pickle('/content/drive/MyDrive/Mofid_Ether_infer/result_'+dataset_name+'.pkl')
            # df_results.index= [i for i in range(len(df_results))]
            # pd.set_option('display.max_rows', len(df_results))
            num_r = 0 
  
            for method in method_list:
              for risk_mode in Risk_modes:
                risk_sensitive_params = get_risk_sensitive_params(risk_mode)
                cnt_rsp = -1 
                for risk_sensitive_param in risk_sensitive_params:
                  cnt_rsp+=1
                  #print('method = ',method)
                  #print('risk_mode = ',risk_mode)
                  last_dco = 1
                  #print('risk_param = ',risk_sensitive_param)
                  if method in ['buy_at_first']:
                    profit = []
                    profit_time_stomps = []
                    trades = []
  
                    num_btc = 0
                    num_dollar = 1000
                    is_first = False
                    for i in range(len(data_bitcoin2015)):
                      current_btc_price = data_bitcoin2015.loc[i,'close']
                      last_btc_price = data_bitcoin2015.loc[i,'open']
                      current_time = str(data_bitcoin2015.loc[i,'date'])
                      # if data_bitcoin2015.loc[i,'close']/data_bitcoin2015.loc[i,'Open']<0.99:
                        # print(data_bitcoin2015.loc[i,'close']/data_bitcoin2015.loc[i,'Open'])
                      # Get information from LSTM
                      if not is_first:
                        # print(data_bitcoin2015)
                      #   print('use loss limit = ',use_loss_limit)
                      #   print('current_time = ',current_time)
                      #   print('buyed with price' + str(last_btc_price))
                        is_first = True
                        num_btc += (1-Transaction_cost_rate)*num_dollar*1/last_btc_price
                        num_dollar = 0
                        trades.append(1)
                        profit.append(num_btc*last_btc_price+num_dollar-1000)
                        profit_time_stomps.append(current_time)
                      else:
                        profit.append(num_btc*last_btc_price+num_dollar-1000)
                        trades.append(0)
                        profit_time_stomps.append(current_time)
                      last_dco = data_bitcoin2015.loc[i,'close']-data_bitcoin2015.loc[i,'open']
                  #   print(last_btc_price)
                  #   print(profit[-1])
                  if method in ['random']:
                    profit = []
                    profit_time_stomps = []
                    trades = []
                    num_btc = 0
                    num_dollar = 1000
                    # is_first = False
                    for i in range(len(data_bitcoin2015)):
                      current_btc_price = data_bitcoin2015.loc[i,'close']
                      last_btc_price = data_bitcoin2015.loc[i,'open']
                      current_time = str(data_bitcoin2015.loc[i,'date'])
                      # Get information from LSTM
                      buy_sell = random.randint(0,2)
                      if buy_sell==0:
                        profit.append(num_btc*last_btc_price+num_dollar-1000)
                        profit_time_stomps.append(current_time)
                        trades.append(0)
                      if buy_sell==1:
                        num_btc += (1-Transaction_cost_rate)*num_dollar/last_btc_price
                        num_dollar -= num_dollar
                        trades.append(1)
                        profit.append(num_btc*last_btc_price+num_dollar-1000)
                        profit_time_stomps.append(current_time)
                      
                      if buy_sell==2:
                        num_dollar += (1-Transaction_cost_rate)*num_btc*last_btc_price
                        num_btc -=num_btc
                        trades.append(-1)
                        profit.append(num_btc*last_btc_price+num_dollar-1000)
                        profit_time_stomps.append(current_time)
                  if method in ['LSTM','LSTM_self_Attention']:
                    profit = []
                    profit_time_stomps = []
                    trades = []
                    buffer = [1,1,1]
                    num_btc = 0
                    last_action = 'sell'
                    last_price = 0
                    num_dollar = 1000
                    for i in range(len(data_bitcoin2015)):
                      current_btc_price = data_bitcoin2015.loc[i,'close']
                      last_btc_price = data_bitcoin2015.loc[i,'open']
                      current_time = str(data_bitcoin2015.loc[i,'date'])
                      btc_price_open = data_bitcoin2015.loc[i,'open']
                      btc_price_high = data_bitcoin2015.loc[i,'high']
                      btc_price_low = data_bitcoin2015.loc[i,'low']
                      btc_price_close = data_bitcoin2015.loc[i,'close']
                      # Get information from LSTM
                      if dataset_type == 'daily':
                        get_item = df_predictions_hourly.loc[df_predictions_hourly['date'].astype('string') == re.search(r'\d{4}-\d{2}-\d{2}', str(data_bitcoin2015.loc[i,'date'])).group()]
                      else:
                        get_item = df_predictions_hourly.loc[df_predictions_hourly['date'].astype('string') == re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(data_bitcoin2015.loc[i,'date'])).group()]
                        
                      acted_ = False
                      # if len(get_item)>0:
                      #   #print(1)
                      #   #print(re.search(r'\d{4}-\d{2}-\d{2}', str(data_bitcoin2015.loc[i,'date'])).group())
                      if last_action == 'buy':
                        if use_loss_limit:
                          if dont_sell_inside_candle:
                            if data_bitcoin2015.loc[i,'open']/last_price < 0.975:
                              acted_ = True
                              num_dollar += (1-Transaction_cost_rate)*num_btc*1*data_bitcoin2015.loc[i,'open']
                              # num_dollar += num_btc*1*max(last_btc_price,current_btc_price)
                              num_btc -=num_btc*1
                              last_action = 'sell'
                              trades.append(-1)
                              profit.append(num_btc*data_bitcoin2015.loc[i,'open']+num_dollar-1000)
                              profit_time_stomps.append(current_time)
                          else:
                            if data_bitcoin2015.loc[i,'close']/last_price < 0.975 and btc_price_low<last_price*0.975<btc_price_high:
                              acted_ = True
                              num_dollar += (1-Transaction_cost_rate)*num_btc*1*last_price*0.975
                              # num_dollar += num_btc*1*max(last_btc_price,current_btc_price)
                              num_btc -=num_btc*1
                              last_action = 'sell'
                              trades.append(-1)
                              profit.append(num_btc*last_btc_price+num_dollar-1000)
                              profit_time_stomps.append(current_time)
                          
                      if not acted_:
                        if len(get_item)>0:
                          # print('1')
                          Estimated_change_ratio_LSTM = float(get_item['Estimated_change_ratio_LSTM'])
                          buy_sell = 0
                          buffer[0] = buffer[1]
                          buffer[1] = buffer[2]
                          buffer[2] = Estimated_change_ratio_LSTM
                          last_price = current_btc_price
                          if Estimated_change_ratio_LSTM>1:
                            last_action = 'buy'
                            buy_sell = 1
                            # print('buy')
                          elif  Estimated_change_ratio_LSTM<1:
                            last_action = 'sell'
                            buy_sell = 2
                          else:
                            buy_sell = 0
                          if len(risk_sensitive_param)==1:
                            # print('intens')
                            buy_sell_vec_intensity = risk_sensitive_func(Estimated_change_ratio_LSTM,risk_sensitive_param[0],risk_mode)
                          
                          if len(risk_sensitive_param)>1:
                            buy_sell_vec_intensity = risk_sensitive_func(Estimated_change_ratio_LSTM,risk_sensitive_param[0],risk_mode,risk_sensitive_param[1])
                          
                          if do_the_new_idea:
                              # buy_sell_vec_intensity = 1
                              nn_sheeb1 = buffer[2] - buffer[1]
                              nn_sheeb2 = buffer[1] - buffer[0]
                              avg = np.mean(buffer)
                              
                              if avg >1 and nn_sheeb1<nn_sheeb2:
                                  buy_sell = 1
                              elif avg <1 and nn_sheeb2<nn_sheeb1:
                                  buy_sell = 2
                              else:
                                  buy_sell = 0
                                                          
                          if buy_sell==0:
                            profit.append(num_btc*last_btc_price+num_dollar-1000)
                            profit_time_stomps.append(current_time)
                            trades.append(0)
                          if buy_sell==1:
                            num_btc += (1-Transaction_cost_rate)*num_dollar*buy_sell_vec_intensity/buy_with_loss_limit(last_dco,btc_price_open,btc_price_close,loss_limit,use_loss_limit)
                            num_dollar -= num_dollar*buy_sell_vec_intensity
                            trades.append(buy_sell_vec_intensity)
                            profit.append(num_btc*last_btc_price+num_dollar-1000)
                            profit_time_stomps.append(current_time)
                          if buy_sell==2:
                            num_dollar += (1-Transaction_cost_rate)*num_btc*buy_sell_vec_intensity*sell_with_loss_limit(last_dco,btc_price_open,btc_price_close,loss_limit,use_loss_limit)
                            # max(btc_price_open,btc_price_close)
                            # buy_with_loss_limit(last_dco,btc_price_open,btc_price_close,loss_limit)
                            num_btc -=num_btc*buy_sell_vec_intensity
                            trades.append(-buy_sell_vec_intensity)
                            profit.append(num_btc*last_btc_price+num_dollar-1000)
                            profit_time_stomps.append(current_time)
                        else:
                          profit.append(num_btc*last_btc_price+num_dollar-1000)
                          profit_time_stomps.append(current_time)
                          trades.append(0)
                      last_dco = data_bitcoin2015.loc[i,'close']-data_bitcoin2015.loc[i,'open']
                  #print(profit[-1])
                  df_results['date'] = profit_time_stomps
                  df_results[method + '_'+str(risk_mode)+'_'+str(cnt_rsp)+'_profit'] = profit
                  df_results[method + '_'+str(risk_mode)+'_'+str(cnt_rsp)+'_tradings'] = trades
                  # df_results['best_acc'] = [best_acc for i in range(len(profit_time_stomps))]
  
            if use_loss_limit:
              # if profiting_interval:
              df_results.to_csv('/content/drive/MyDrive/long_term_simulation_with_loss_bound_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv')
              # if not profiting_interval:
              #   df_results.to_csv('/content/drive/MyDrive/long_term_simulation_with_loss_bound_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_non_profiting'+'.csv')    
            else:
              # if profiting_interval:
              df_results.to_csv('/content/drive/MyDrive/long_term_simulation_without_loss_bound_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_'+mode_dataset_strip+'_task_1'+'.csv')
              # if not profiting_interval:
              #   df_results.to_csv('/content/drive/MyDrive/long_term_simulation_without_loss_bound_'+dataset_type+'_'+crypto_type+'_'+str(observe_delay)+'_non_profiting'+'.csv')

round  1
mode_dataset_strip  remove_first_month
dataset_type  daily
observe_delay  
resume incomplete process
end_train_time =  2021-10-13 00:00:00
ender_time =  2021-11-13 00:00:00
starting the predictions
starting the trading


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
